In [14]:
import tweepy
import json
import nltk
from nltk.collocations import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import re

In [9]:
#API Keys
CONSUMER_KEY = 'NDftFMzyOVTqKZf4TkzpJAN8E'
CONSUMER_SECRET = 'YuvOwUZ6dytuMk52mw7eqcPuSKdvkelaziWOo6w2W7kGcGdE9X'

#Access Tokens
OAUTH_TOKEN = '1244759591365328896-5ghELsW3rwRfvPK90GtcQ3aeO23qGz'
OAUTH_SECRET = 'gZXQVyHxjnHTeV9ao7ybiOAfNX2akBRulwzh3I9lRXNfB'

In [10]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth)

## Q1: Process one collection of data and summarize information from several fields.

In [11]:
# search for tweets that contain "Biden"
query = "Biden"
date_since = "2020-1-1"
Biden_tweets = [status for status in tweepy.Cursor(api.search, q = query, since=date_since).items(500)]

### A. change all 500 json tweets to a dataframe

In [44]:
users_locs = [[tweet.created_at,tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in Biden_tweets]
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['created_at', "user_name","location","text"])
tweet_text.head()

,created_at,user_name,location,text
0,2020-11-24 04:29:46,AGarblek,,RT @FragoliAlex: @robinmonotti For anyone watc...
1,2020-11-24 04:29:46,itsowiener,,"RT @OnlyInVegas702: Wait, so Trump passed a bi..."
2,2020-11-24 04:29:46,LMX0740,,RT @ejmaroun: Biden’s won in Michigan so many ...
3,2020-11-24 04:29:45,WilliamJacinski,"Ponta Grossa, Brasil",@gugachacra Parece que certo jornalista está t...
4,2020-11-24 04:29:45,sylva2sly,"Texas, USA",Heee 😀🤣🤣. This is America. Trump vs Biden! Lol...


In [48]:
import os
os.getcwd()

'/Users/sijinzhou/Desktop'

In [50]:
tweet_text.to_csv('/Users/sijinzhou/Desktop//tweet_text_summary.csv')

### B. Analyze the hashtag of those tweets

In [52]:
# get hashtags of those tweets
biden_hashtags = [doc._json['entities']['hashtags'] for doc in Biden_tweets]
biden_hashtags = [hashtag for hashtag in biden_hashtags if hashtag]
print(f"There are {len(biden_hashtags)} hashtags among these 500 tweets.")

There are 23 hashtags among these 500 tweets.


In [53]:
# extrat a list of lower case hashtag text
hashtagtext = []
for item in biden_hashtags:
    for text in item:
        hashtagtext.append(text['text'].lower())

In [54]:
#since hashatag is the already key words of the tweets, 
#I have no need to filter stopwords. 
msgFD = nltk.FreqDist(hashtagtext)
t = msgFD.most_common(20)

hashtag_word = []
hashtag_wordcount = []
for a, b in msgFD.most_common(10):
    hashtag_word.append(a)
    hashtag_wordcount.append(b)

In [55]:
#convert it to data frame
hashtag_wordcount_df = pd.DataFrame(
    {'hashtag_word': hashtag_word,
     'hashtag_wordcount': hashtag_wordcount
    })
hashtag_wordcount_df

,hashtag_word,hashtag_wordcount
0,trump,4
1,biden,4
2,dnc,2
3,maddow,2
4,breaking,2
5,suxtobedon,1
6,dasvidaniyadon,1
7,ccp,1
8,msm,1
9,bbcnews,1


In [56]:
hashtag_wordcount_df.to_csv('/Users/sijinzhou/Desktop//hashtag_wordcount.csv')

### C. Analyze text of the thoses tweets

In [58]:
# get the text field for the tweets lsit
biden_texts = [doc._json['text'] for doc in Biden_tweets]

In [59]:
# initialize a TweetTokenizer
ttokenizer = nltk.tokenize.TweetTokenizer()

In [60]:
#tokenize the elemtent using the twitter tokenizer
all_biden_tokens = [tok.lower() for tweet in biden_texts for tok in ttokenizer.tokenize(tweet)]

In [61]:
#import the collocation finder module
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [62]:
finder = BigramCollocationFinder.from_words(all_biden_tokens)

In [63]:
import re
# function that takes a word and returns true if it consists only of non-alphabetic characters
def alpha_filter(w):
    # pattern to match a word of non-alphabetical characters
        # ^ - start of string or line
        # [^ ] - not
        # a-z - lowercase a-z
        # + - one or more 
        # $ - end of string        
    pattern = re.compile('^[^a-z]+$')
    if (pattern.match(w)):
        return True
    else:
        return False

In [64]:
#filter out the special characters by using the alpha_filter function
finder.apply_word_filter(alpha_filter)

In [65]:
# stopwords list in NLTK for English
nltk_stopwords = nltk.corpus.stopwords.words('english')
# add "rt" into the stopwords list
nltk_stopwords.append('rt')
len(nltk_stopwords)

180

In [66]:
#filter out the stopwords
finder.apply_word_filter(lambda w: w in nltk_stopwords)

In [67]:
#scoring the bigrams by frequency
scored = finder.score_ngrams(bigram_measures.raw_freq)

word = []
score = []
for a, b in scored[:30]:
    word.append(a)
    score.append(b)
scored[:10]

[(('joe', 'biden'), 0.01043270053018642),
 (('president-elect', 'joe'), 0.002223362408072516),
 (('open', 'skies'), 0.001624764836668377),
 (('president-elect', 'biden'), 0.0015392508978963571),
 (('white', 'house'), 0.0015392508978963571),
 (('donald', 'trump'), 0.0014537369591243372),
 (('formal', 'transition'), 0.0013682230203523175),
 (('skies', 'treaty'), 0.0012827090815802976),
 (('national', 'security'), 0.0011971951428082776),
 (('top', 'story'), 0.0011971951428082776)]

In [68]:
# import bigram score as dataframe
wordcount_df = pd.DataFrame(
    {'word': word,
     'score': score
    })

In [69]:
wordcount_df.head(10)

,word,score
0,"(joe, biden)",0.010433
1,"(president-elect, joe)",0.002223
2,"(open, skies)",0.001625
3,"(president-elect, biden)",0.001539
4,"(white, house)",0.001539
5,"(donald, trump)",0.001454
6,"(formal, transition)",0.001368
7,"(skies, treaty)",0.001283
8,"(national, security)",0.001197
9,"(top, story)",0.001197


In [70]:
wordcount_df.to_csv('/Users/sijinzhou/Desktop//wordcount_df.csv')

### D. Analyze sentiment in tweets

In [4]:
#install textblob in mac
conda install -c conda-forge/label/cf202003 textblob

Solving environment: done

## Package Plan ##

  environment location: /Users/sijinzhou/opt/anaconda3

  added / updated specs:
    - textblob


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge/label/cf202003
    textblob-0.15.3            |             py_0         595 KB  conda-forge/label/cf202003
    ------------------------------------------------------------
                                           Total:         743 KB

The following NEW packages will be INSTALLED:

  textblob           conda-forge/label/cf202003/noarch::textblob-0.15.3-py_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                         pkgs/main --> conda-forge/label/cf202003



textblob-0.15.3      | 595 KB    | ##################################### | 100% 
certifi-2019.11.28   

In [5]:
from textblob import TextBlob

In [6]:
#remove url to prepare for sentment analysis
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [15]:
tweets_no_urls = [remove_url(tweet.text) for tweet in Biden_tweets]

In [16]:
# Create textblob objects of the tweets
sentiment_objects = [TextBlob(tweet) for tweet in tweets_no_urls]
sentiment_objects[0].polarity, sentiment_objects[0]

(0.0,
 TextBlob("RT FragoliAlex robinmonotti For anyone watching in the USA be in no doubt of what awaits you if Biden becomes president We are seeing"))

In [17]:
# Create list of polarity valuesx and tweet text
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]

sentiment_values[0]

[0.0,
 'RT FragoliAlex robinmonotti For anyone watching in the USA be in no doubt of what awaits you if Biden becomes president We are seeing']

In [18]:
# Create dataframe containing the polarity value and tweet text
sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])

sentiment_df.head()

,polarity,tweet
0,0.00,RT FragoliAlex robinmonotti For anyone watchin...
1,0.00,RT OnlyInVegas702 Wait so Trump passed a bill ...
2,0.35,RT ejmaroun Bidens won in Michigan so many tim...
3,0.00,gugachacra Parece que certo jornalista est ten...
4,0.80,Heee This is America Trump vs Biden Lol


In [71]:
sentiment_df.to_csv('/Users/sijinzhou/Desktop//sentiment_df.csv')

## Q2:process one collection of data and separate it into different categories and provided summary statistics on those categories. 

In [25]:
#collect Biden timelines 
biden_list=[]
for status in tweepy.Cursor(api.user_timeline, screen_name='JoeBiden',tweet_mode="extended").items(500):
    biden_list.append(status)

In [26]:
# fetch the time each each Biden tweet was created and extract the hour
biden_hour_df = pd.DataFrame()

import datetime
timelist=[]
for doc in biden_list:
    row = [doc.created_at, doc.created_at.hour]
    df = pd.DataFrame([row],columns = ['time','hour'])
    biden_hour_df = biden_hour_df.append(df)

        
biden_hour_df.head()

,time,hour
0,2020-11-24 03:05:01,3
0,2020-11-24 02:47:00,2
0,2020-11-23 20:13:00,20
0,2020-11-23 17:24:23,17
0,2020-11-23 14:30:00,14


### Bin the tweets by hour

In [27]:
# bin Biden tweets by hour
biden_groupby_hour = biden_hour_df.groupby(['hour']).count()
biden_groupby_hour

,time
hour,
0,28
1,44
2,36
3,24
4,14
5,10
6,5
7,1
11,6


In [72]:
biden_groupby_hour.to_csv('/Users/sijinzhou/Desktop//biden_groupby_hour.csv')

In [29]:
# fetch the time each each Biden tweet was created and extract the hour
biden_day_df = pd.DataFrame()

import datetime
timelist=[]
for doc in biden_list:
    row = [doc.created_at, doc.created_at.day]
    df = pd.DataFrame([row],columns = ['time','day'])
    biden_day_df = biden_day_df.append(df)

        
biden_day_df.head()

,time,day
0,2020-11-24 03:05:01,24
0,2020-11-24 02:47:00,24
0,2020-11-23 20:13:00,23
0,2020-11-23 17:24:23,23
0,2020-11-23 14:30:00,23


In [31]:
# bin Biden tweets by day
biden_groupby_day = biden_day_df.groupby(['day']).count()
biden_groupby_day

,time
day,
1,49
2,53
3,54
4,22
5,13
6,3
7,9
8,16
9,8


In [73]:
biden_groupby_day.to_csv('/Users/sijinzhou/Desktop//biden_groupby_day.csv')

## Q3: Process two or more collections of data and compare some summary data about the two collections together.

In [33]:
#collect Biden timelines 
biden_list=[]
for status in tweepy.Cursor(api.user_timeline, screen_name='JoeBiden',tweet_mode="extended").items(500):
    biden_list.append(status)

In [32]:
#collect Trump timelines 
trump_list=[]
for status in tweepy.Cursor(api.user_timeline, screen_name='realDonaldTrump',tweet_mode="extended").items(500):
    trump_list.append(status)

### compare the number of their followers, friends, public lists, favorites, times of their tweets been retweets and favorited

In [34]:
# get'followers_count','friends_count','listed_count','favourites_count' of JoeBiden
biden = api.get_user('JoeBiden')
type(biden)

tweepy.models.User

In [35]:
key = ['followers_count','friends_count','listed_count','favourites_count']
biden_value = [biden._json[k] for k in key]
biden_value

[19618285, 30, 29185, 20]

In [36]:
# get'followers_count','friends_count','listed_count','favourites_count' of Trump
trump = api.get_user('realDonaldTrump')
type(trump)

tweepy.models.User

In [37]:
trump_value = [trump._json[k] for k in key]
trump_value

[88904921, 51, 125256, 6]

In [39]:
#sum retweet
biden_retweet = [x.retweet_count for x in biden_list]
trump_retweet = [x.retweet_count for x in trump_list]
print(f"In total, these 500 Biden tweets get {sum(biden_retweet)} retweets, while Trump get {sum(trump_retweet)} retweets")

In total, these 500 Biden tweets get 8665498 retweets, while Trump get 2424493 retweets


In [40]:
#sum favorite
biden_favorite = [x.favorite_count for x in biden_list]
trump_favorite = [x.favorite_count for x in trump_list]
print(f"In total, these 500 Biden tweets get {sum(biden_favorite)} favorite, while Trump get {sum(trump_favorite)} favorite")

In total, these 500 Biden tweets get 73913753 favorite, while Trump get 9829598 favorite


In [41]:
# append retweet and favorite to previous value list
biden_value.append(sum(biden_retweet))
trump_value.append(sum(trump_retweet))

biden_value.append(sum(biden_favorite))
trump_value.append(sum(trump_favorite))
biden_value

[19618285, 30, 29185, 20, 8665498, 73913753]

In [42]:
# convert into data frame
summary_df = pd.DataFrame({
    'Biden':biden_value,
    'Trump':trump_value},
    index=['followers_count','friends_count','listed_count','favourites_count','retweet','favorite'])
summary_df

,Biden,Trump
followers_count,19618285,88904921
friends_count,30,51
listed_count,29185,125256
favourites_count,20,6
retweet,8665498,2424493
favorite,73913753,9829598


In [75]:
summary_df.to_csv('/Users/sijinzhou/Desktop//Biden_trump_summary.csv')